In [72]:
import pandas as pd

In [77]:
spending = pd.read_csv('spending_2024.csv', delimiter=';')

In [78]:
spending.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 368 entries, 0 to 367
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Date         368 non-null    object 
 1   Description  368 non-null    object 
 2   Amount       337 non-null    float64
dtypes: float64(1), object(2)
memory usage: 8.8+ KB


In [79]:
spending.drop(spending.loc[spending['Description'].str.contains(r'PAY CIUSSS DE L\' EST DE MONTREAL', na=False)].index, inplace=True) # Need to ignore paycheck
spending = spending.dropna() # Remove rows with null values

In [80]:
spending.dropna(inplace=True) # Remove all the null values
print(spending.isna().sum()) # Checking if there is any lines with null values

Date           0
Description    0
Amount         0
dtype: int64


In [81]:
substring = 'RETAIL PURCHASE'
spending['Description'] = spending['Description'].apply(lambda line:line[line.find(substring):] if substring in line else line)

In [83]:


def remove_numeric(substring):
    if substring.isnumeric() and len(substring) == 12:
        return ''
    return substring

spending['Description'] = spending['Description'].apply(
    lambda line: " ".join(remove_numeric(word) for word in line.split())
)


0                        RETAIL PURCHASE SH VENDING80036
1                        RETAIL PURCHASE SUPER C PTE AUX
2                        RETAIL PURCHASE SH VENDING80036
3                        RETAIL PURCHASE HOPITAL MAISONN
4      Electronic Funds Transfer PREAUTHORIZED DEBIT ...
                             ...                        
363                      RETAIL PURCHASE DEPANNEUR TRICE
364                       RETAIL PURCHASE A+ PRODUITS DE
365                      RETAIL PURCHASE DOLLARAMA # 130
366                      RETAIL PURCHASE WINNERS/HOMESEN
367                         RETAIL PURCHASE AMZN Mktp CA
Name: Description, Length: 337, dtype: object


In [86]:
spending.head(5)

,Date,Description,Amount
0,2024-06-28,RETAIL PURCHASE SH VENDING80036,1.85
1,2024-06-27,RETAIL PURCHASE SUPER C PTE AUX,28.19
2,2024-06-27,RETAIL PURCHASE SH VENDING80036,3.15
3,2024-06-26,RETAIL PURCHASE HOPITAL MAISONN,4.40
4,2024-06-25,Electronic Funds Transfer PREAUTHORIZED DEBIT ...,18.97
